VI.9.32

| Год         | 1910         | 1920        | 1930        | 1940        | 1950        |
| -------     | ------------ | ----------- | ----------- | ----------- | ----------- |
| Население   | 092 228 496  | 106 021 537 | 123 202 624 | 132 164 569 | 151 325 798 |

| Год         | 1960         | 1970        | 1980        | 1990        | 2000        |
| -------     | ------------ | ----------- | ----------- | ----------- | ----------- |
| Население   | 179 323 175  | 203 211 926 | 226 545 805 | 248 709 873 | 281 421 906 |


In [370]:
import numpy as np

years = np.array([1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000])
population = np.array([92228496, 106021537, 123202624, 132164569, 151325798, 179323175, 203211926, 226545805, 248709873, 281421906])

а) Построить интерполянт в форме Ньютона. Вычислить экстраполированное значение численности населения в 2010 году и сравнить с точным значением 308 745 538.

Функция для вычисления разделённой разности:

In [371]:
def sep_diff(years, population):
  sz = np.shape(years)[0]
  sp_tbl = np.zeros([sz, sz])
  for i in range (0, sz):
    sp_tbl[i, 0] = population[i]
  for j in range (1, sz):
    for i in range (1, sz):
      sp_tbl[i, j] = (sp_tbl[i, j - 1] - sp_tbl[i-1, j-1]) / (years[i] - years[i - j])
  
  res = np.zeros([sz])
  for i in range(0, sz):
    res[i] = sp_tbl[i, i]
  
  return res

Функция для вычисления полинома Ньютона. Общий вид формулы выглядит так:

$$N_n(x) = b_0 + b_1\cdot(x-x_0) + b_2\cdot(x-x_0)\cdot(x-x_1) + ... + b_n\cdot(x-x_0)\cdot(x-x_1)\cdot...\cdot(x-x_n)$$

In [372]:
def poly_Newton(years, population, x):
  sp_tbl = sep_diff(years, population)
  sz = np.shape(sp_tbl)[0]
  aux_poly_val = 1
  res = sp_tbl[0]
  for i in range (sz - 1):
    aux_poly_val = aux_poly_val * (x - years[i])
    res = res + sp_tbl[i + 1] * aux_poly_val

  return res

Вычисление значения в 2010 году:

In [373]:
res = poly_Newton(years, population, 2010)
expected_res = 308745538
error = int(np.abs(res / expected_res - 1) * 100)

print("Полученное значение полинома Ньютона:", int(res))
print("Ожидаемое значение полинома Ньютона: ", expected_res)
print("Погрешность вычисления:", error, "%")

Полученное значение полинома Ньютона: 827906509
Ожидаемое значение полинома Ньютона:  308745538
Погрешность вычисления: 168 %


б) Построить сплайн-аппроксимацию, экстраполировать данные на 2010 год.

Используем сплайн:

$$S_k = a_k + b_k (x-x_k) + \frac{c_k}{2} (x-x_k)^2 + \frac{d_k}{6} (x-x_k)^3$$

Также поставим дополнительные условия:

$$f''(1910) = 0 = f''(2000)$$

In [374]:
import bisect

def calc_diffs(x):
  sz = len(x) - 1
  res = np.zeros(sz)
  for i in range (sz):
    res[i] = x[i+1] - x[i]

  return res

def tridiag_init(n, h):
  A = np.zeros(n - 1)
  C = np.zeros(n - 1)
  for i in range (n - 2):
    A[i] = h[i] / (h[i] + h[i + 1])
    C[i + 1] = h[i] / (h[i] + h[i + 1])

  B = np.zeros(n)
  for i in range (n):
    B[i] = 2
  
  return A, B, C

def target_init(n, h, y):
  res_arr = np.zeros(n)
  for i in range (1, n-1):
    res_arr[i] = 6 * ((y[i + 1] - y[i]) / h[i] - (y[i] - y[i - 1]) / h[i - 1]) / (h[i] + h[i-1])

  return res_arr

def calc_tridiag(A, B, C, D):
  c_p = np.append(C, 0)
  d_p = np.zeros(np.shape(B)[0])
  res = np.zeros(np.shape(B)[0])
  for i in range(0, np.shape(B)[0]):
    if i == 0:
      c_p[i] = C[i] / B[i]
      d_p[i] = D[i] / B[i]
    else:
      c_p[i] = c_p[i] / (B[i] - c_p[i - 1] * A[i - 1])
      d_p[i] = (D[i] - d_p[i - 1] * A[i - 1]) / (B[i] - c_p[i - 1] * A[i - 1])

  res[-1] = d_p[-1]
  for i in range(np.shape(B)[0] - 2, -1, -1):
      res[i] = d_p[i] - c_p[i] * res[i + 1]

  return res

def calc_spline(x, y, num):
  n = len(x)
  h = calc_diffs(x)
  A, B, C = tridiag_init(n, h)
  D = target_init(n, h, y)
  tridiag_solved = calc_tridiag(A, B, C, D)

  cfs = np.array([[(tridiag_solved[i+1]-tridiag_solved[i]) * h[i] * h[i] / 6, tridiag_solved[i] * h[i] * h[i] / 2, (y[i + 1] - y[i] - (tridiag_solved[i + 1]+  2 * tridiag_solved[i]) * h[i] * h[i] / 6), y[i]] for i in range(n - 1)])

  idx = min(bisect.bisect(x, num)-1, n-2)
  z = (num - x[idx]) / h[idx]
  C = cfs[idx]
  return (((C[0] * z) + C[1]) * z + C[2]) * z + C[3]



Вычислим значение, которое мы получим с помощью сплайн-аппроксимации.

In [375]:
spline_res = calc_spline(years, population, 2010)
expected_res = 308745538
error = int(np.abs(spline_res / expected_res - 1) * 100)

print("Полученное значение сплайн-аппроксимации:", int(spline_res))
print("Ожидаемое значение: ", expected_res)
print("Погрешность вычисления:", error, "%")

Полученное значение сплайн-аппроксимации: 314133939
Ожидаемое значение:  308745538
Погрешность вычисления: 1 %
